<img src="https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png" width="50" height="50">

# Homework 4 - Political Polling

## Part 0: Imports
In the below cell, please import all modules you need. You may come back to this cell to add new modules you've discovered you might need. **The only way to get this part wrong is to have imports in later cells in this notebook**.

In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
import sklearn.metrics as mt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

## Part 1: Read in data
Explore it, too. Clean anything you need to.

In [2]:
va_pol = pd.read_csv('data/polling-va-05.csv')
va_pol.head()

,response,ager,educ,file_race,gender,race_eth,approve,genballot,hdem_fav,hrep_fav,age_combined,educ4,file_party,gender_combined,likely,partyid,region,turnout_class,phone_type
0,Rep,50 to 64,Graduate or Professional Degree,Unknown,Female,Hispanic,Approve,Reps. keep House,Unfavorable,Favorable,45 to 64,Postgraduate Degree,Republican,Female,Very likely,Independent (No party),North,85 percent or higher,Cell
1,Rep,18 to 34,Some college or trade school,Unknown,Male,White,Approve,Reps. keep House,Don't know,Don't know,18 to 29,Some College Educ.,Other,Male,Somewhat likely,Republican,South,Less than 50 percent,Cell
2,Dem,50 to 64,Graduate or Professional Degree,White,Female,White,Disapp.,Dems. take House,Favorable,Unfavorable,45 to 64,Postgraduate Degree,Democratic,Female,Almost certain,Democrat,Charlottesville,85 percent or higher,Cell
3,Rep,65 and older,Graduate or Professional Degree,White,Female,White,Approve,Reps. keep House,Unfavorable,Favorable,65 and older,Postgraduate Degree,Republican,Female,Almost certain,Republican,South,85 percent or higher,Cell
4,Dem,18 to 34,Bachelors' degree,White,Male,Other,Approve,Dems. take House,Don't know,Don't know,18 to 29,4-year College Grad.,Democratic,Male,Almost certain,Democrat,South,50 to 85 percent,Cell


In [3]:
va_pol.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501 entries, 0 to 500
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   response         501 non-null    object
 1   ager             501 non-null    object
 2   educ             501 non-null    object
 3   file_race        501 non-null    object
 4   gender           501 non-null    object
 5   race_eth         501 non-null    object
 6   approve          501 non-null    object
 7   genballot        501 non-null    object
 8   hdem_fav         501 non-null    object
 9   hrep_fav         501 non-null    object
 10  age_combined     501 non-null    object
 11  educ4            501 non-null    object
 12  file_party       501 non-null    object
 13  gender_combined  501 non-null    object
 14  likely           501 non-null    object
 15  partyid          501 non-null    object
 16  region           501 non-null    object
 17  turnout_class    501 non-null    ob

In [4]:
va_pol.replace("[DO NOT READ] Don't know/Refused", np.NaN, inplace=True)

In [5]:
va_pol.replace("[DO NOT READ] Refused", np.NaN, inplace=True)

In [6]:
va_pol.isnull().sum(axis = 0)

response            0
ager               25
educ                4
file_race           0
gender              0
race_eth            6
approve             0
genballot           0
hdem_fav            0
hrep_fav            0
age_combined        0
educ4               4
file_party          0
gender_combined     0
likely              6
partyid            15
region              0
turnout_class       0
phone_type          0
dtype: int64

In [7]:
va_pol.groupby(['race_eth'])["file_race"].value_counts()

race_eth  file_race
Asian     Asian          1
          Other          1
          White          1
Black     Black         42
          White         26
          Unknown        2
Hispanic  White         11
          Black          3
          Hispanic       3
          Unknown        1
Other     White          8
          Unknown        2
          Black          1
White     White        332
          Unknown       30
          Black         17
          Hispanic       8
          Asian          3
          Other          3
Name: file_race, dtype: int64

In [8]:
pol = va_pol.drop(['ager', 'file_race', 'educ'], axis=1)
pol.head()

,response,gender,race_eth,approve,genballot,hdem_fav,hrep_fav,age_combined,educ4,file_party,gender_combined,likely,partyid,region,turnout_class,phone_type
0,Rep,Female,Hispanic,Approve,Reps. keep House,Unfavorable,Favorable,45 to 64,Postgraduate Degree,Republican,Female,Very likely,Independent (No party),North,85 percent or higher,Cell
1,Rep,Male,White,Approve,Reps. keep House,Don't know,Don't know,18 to 29,Some College Educ.,Other,Male,Somewhat likely,Republican,South,Less than 50 percent,Cell
2,Dem,Female,White,Disapp.,Dems. take House,Favorable,Unfavorable,45 to 64,Postgraduate Degree,Democratic,Female,Almost certain,Democrat,Charlottesville,85 percent or higher,Cell
3,Rep,Female,White,Approve,Reps. keep House,Unfavorable,Favorable,65 and older,Postgraduate Degree,Republican,Female,Almost certain,Republican,South,85 percent or higher,Cell
4,Dem,Male,Other,Approve,Dems. take House,Don't know,Don't know,18 to 29,4-year College Grad.,Democratic,Male,Almost certain,Democrat,South,50 to 85 percent,Cell


In [9]:
pol.isnull().sum(axis = 0)

response            0
gender              0
race_eth            6
approve             0
genballot           0
hdem_fav            0
hrep_fav            0
age_combined        0
educ4               4
file_party          0
gender_combined     0
likely              6
partyid            15
region              0
turnout_class       0
phone_type          0
dtype: int64

In [10]:
va_pol.groupby(['file_party'])["partyid"].value_counts(dropna=False)

file_party  partyid                                  
Democratic  Democrat                                     76
            Independent (No party)                       36
            NaN                                           3
            Republican                                    3
Other       Independent (No party)                       75
            Democrat                                     63
            Republican                                   54
            NaN                                          10
            or as a member of another political party     4
Republican  Republican                                   96
            Independent (No party)                       59
            Democrat                                     11
            or as a member of another political party     9
            NaN                                           2
Name: partyid, dtype: int64

In [11]:
#if reported party ID is unknown/refused, use file party ID

pol.partyid.fillna(pol.file_party, inplace=True)

In [12]:
pol.groupby(['file_party'])["partyid"].value_counts(dropna=False)

file_party  partyid                                  
Democratic  Democrat                                     76
            Independent (No party)                       36
            Democratic                                    3
            Republican                                    3
Other       Independent (No party)                       75
            Democrat                                     63
            Republican                                   54
            Other                                        10
            or as a member of another political party     4
Republican  Republican                                   98
            Independent (No party)                       59
            Democrat                                     11
            or as a member of another political party     9
Name: partyid, dtype: int64

In [13]:
pol[pol.isna().any(axis=1)].shape

(13, 16)

In [14]:
pol.dropna(inplace=True)

In [15]:
pol[pol.isna().any(axis=1)].shape

(0, 16)

In [16]:
pol.groupby(['gender_combined'])["gender"].value_counts(dropna=False)

gender_combined  gender
Female           Female    273
                 Male        3
Male             Male      208
                 Female      4
Name: gender, dtype: int64

In [17]:
pol.response.value_counts()

Dem    220
Rep    220
Und     48
Name: response, dtype: int64

In [18]:
pol.approve.value_counts()

Approve       243
Disapp.       217
Don't know     28
Name: approve, dtype: int64

In [19]:
pol.hrep_fav.value_counts()

Favorable      189
Unfavorable    167
Don't know     132
Name: hrep_fav, dtype: int64

In [20]:
pol.age_combined.value_counts()

45 to 64        198
65 and older    186
30 to 44         74
18 to 29         30
Name: age_combined, dtype: int64

In [21]:
pol.educ4.value_counts()

Some College Educ.           159
Postgraduate Degree          120
4-year College Grad.         108
High School Grad. or Less    101
Name: educ4, dtype: int64

In [22]:
pol.likely.value_counts()

Almost certain       306
Very likely          124
Somewhat likely       22
Not very likely       15
Not at all likely     11
Already voted         10
Name: likely, dtype: int64

In [23]:
pol.partyid.value_counts()

Independent (No party)                       167
Republican                                   155
Democrat                                     145
or as a member of another political party     12
Other                                          8
Democratic                                     1
Name: partyid, dtype: int64

In [24]:
#cleaning up party ID variables

party = {'Independent (No party)':'Other', 'or as a member of another political party':'Other', 'Democratic':'Democrat'}
pol.partyid.replace(party, inplace=True)
pol.partyid.value_counts()

Other         187
Republican    155
Democrat      146
Name: partyid, dtype: int64

In [25]:
8+167+12

187

In [26]:
pol.region.value_counts()

South              284
Charlottesville    103
North              101
Name: region, dtype: int64

In [27]:
pol.genballot.value_counts()

Reps. keep House    239
Dems. take House    219
Don't know           30
Name: genballot, dtype: int64

In [28]:
pol.response.value_counts()

Dem    220
Rep    220
Und     48
Name: response, dtype: int64

## Part 2: Dummy Encoding
Dummify all categorical variables from the previous part that you want to use in the next part. In this step, you should also label encode your y-variable (ie. map values to 0/1/2) and create your $\mathbf{X}$ and $\mathbf{y}$ variables.

In [29]:
pol.columns

Index(['response', 'gender', 'race_eth', 'approve', 'genballot', 'hdem_fav',
       'hrep_fav', 'age_combined', 'educ4', 'file_party', 'gender_combined',
       'likely', 'partyid', 'region', 'turnout_class', 'phone_type'],
      dtype='object')

In [30]:
# y = response
# x = gender, race_eth, approve, genballot, hdem_fav, hrep_fav, age_combined, educ4, likely, partyid, region

In [31]:
# create dummy variables


gender_dum = pd.get_dummies(pol.gender, drop_first=True) # 0 = Female 
race = pd.get_dummies(pol.race_eth, prefix='race', drop_first=True) # 0 = White
is_approve = pd.get_dummies(pol.approve, prefix='approve', drop_first=True) # 0 = Approves (of Republican incumbent?)
predicts_dem_win = pd.get_dummies(pol.genballot, prefix='genballot', drop_first=True) # 0 = predicts Republicans will keep the House
dislikes_dems = pd.get_dummies(pol.hdem_fav, prefix='fav_dem', drop_first=True) # 0 = Favorable view of Democrats
dislikes_reps = pd.get_dummies(pol.hrep_fav, prefix='fav_rep', drop_first=True) # 0 = Favorable view of Republicans
party_affil = pd.get_dummies(pol.partyid, prefix='party', drop_first=True) # 0 = Independent, 1 = Republican, 2 = Democrat
region_d = pd.get_dummies(pol.region, prefix='region', drop_first=True) # 0 = South, 1 = Charlottesville, 2 = North
vote_rep = pd.get_dummies(pol.response, prefix='response', drop_first=True)

In [32]:
# encode other categorical variables

age_grp = {'18 to 29':0, '30 to 44':1, '45 to 64':2, '65 and older':3} # value increasing with age
edu_grp = {'High School Grad. or Less':0, 'Some College Educ.':1, '4-year College Grad.':2, 'Postgraduate Degree':3} # increasing with education
likely_grp = {'Not at all likely':0, 'Not very likely':1, 'Somewhat likely':2, 'Very likely':3, 'Almost certain':4, 'Already voted':5} # increasing with likelihood of voting

pol.age_combined.replace(age_grp, inplace=True)
pol.educ4.replace(edu_grp, inplace=True)
pol.likely.replace(likely_grp, inplace=True)

In [64]:
pol

,response,gender,race_eth,approve,genballot,hdem_fav,hrep_fav,age_combined,educ4,file_party,...,fav_dem_Favorable,fav_dem_Unfavorable,fav_rep_Favorable,fav_rep_Unfavorable,party_Other,party_Republican,region_North,region_South,response_Rep,response_Und
0,Rep,Female,Hispanic,Approve,Reps. keep House,Unfavorable,Favorable,2,3,Republican,...,0,1,1,0,1,0,1,0,1,0
1,Rep,Male,White,Approve,Reps. keep House,Don't know,Don't know,0,1,Other,...,0,0,0,0,0,1,0,1,1,0
2,Dem,Female,White,Disapp.,Dems. take House,Favorable,Unfavorable,2,3,Democratic,...,1,0,0,1,0,0,0,0,0,0
3,Rep,Female,White,Approve,Reps. keep House,Unfavorable,Favorable,3,3,Republican,...,0,1,1,0,0,1,0,1,1,0
4,Dem,Male,Other,Approve,Dems. take House,Don't know,Don't know,0,2,Democratic,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,Rep,Male,White,Approve,Reps. keep House,Don't know,Favorable,3,0,Republican,...,0,0,1,0,0,1,0,1,1,0
497,Dem,Male,White,Disapp.,Don't know,Don't know,Don't know,1,3,Republican,...,0,0,0,0,1,0,0,0,0,0
498,Rep,Female,White,Approve,Reps. keep House,Favorable,Don't know,3,0,Republican,...,1,0,0,0,0,1,1,0,1,0
499,Dem,Female,White,Disapp.,Dems. take House,Favorable,Unfavorable,3,2,Republican,...,1,0,0,1,1,0,0,0,0,0


In [34]:
new_vars = [gender_dum, race, is_approve, predicts_dem_win, dislikes_dems, dislikes_reps, party_affil, region_d, vote_rep]

for var in new_vars:
    pol = pd.concat([pol, var], axis=1)


In [35]:
pol.columns

Index(['response', 'gender', 'race_eth', 'approve', 'genballot', 'hdem_fav',
       'hrep_fav', 'age_combined', 'educ4', 'file_party', 'gender_combined',
       'likely', 'partyid', 'region', 'turnout_class', 'phone_type', 'Male',
       'race_Black', 'race_Hispanic', 'race_Other', 'race_White',
       'approve_Disapp.', 'approve_Don't know', 'genballot_Don't know',
       'genballot_Reps. keep House', 'fav_dem_Favorable',
       'fav_dem_Unfavorable', 'fav_rep_Favorable', 'fav_rep_Unfavorable',
       'party_Other', 'party_Republican', 'region_North', 'region_South',
       'response_Rep', 'response_Und'],
      dtype='object')

In [36]:
pol.race_eth.value_counts()

White       389
Black        67
Hispanic     18
Other        11
Asian         3
Name: race_eth, dtype: int64

In [37]:
features = ['age_combined', 'educ4', 'likely', 'Male', 'race_Black',
            'race_Hispanic', 'race_Other', 'race_White', 'approve_Disapp.',
            "approve_Don't know", "genballot_Don't know", 'genballot_Reps. keep House',
            'fav_dem_Favorable', 'fav_dem_Unfavorable', 'fav_rep_Favorable', 'fav_rep_Unfavorable',
            'party_Other', 'party_Republican', 'region_North', 'region_South']
X = pol[features]
y = pol.response_Rep

## Part 3: Fit a Random Forest Model to Predict Poll Response
No need to do any fancing tuning, unless you want to.

In [38]:
rfreg = RandomForestRegressor(n_estimators=150,
                              max_features=5,
                              oob_score=True)
rfreg.fit(X, y)

RandomForestRegressor(max_features=5, n_estimators=150, oob_score=True)

In [39]:
pd.DataFrame({'feature':features,
              'importance':rfreg.feature_importances_}).sort_values(by='importance')

,feature,importance
5,race_Hispanic,0.000978
6,race_Other,0.001436
18,region_North,0.005714
7,race_White,0.008221
16,party_Other,0.009826
10,genballot_Don't know,0.010038
4,race_Black,0.011802
19,region_South,0.012007
9,approve_Don't know,0.012287
3,Male,0.019869


## Part 4: Evaluation

### Part 4A: What is the baseline accuracy of your model? Did your OOB score beat it?

In [42]:
print(y.mean())
print((rfreg.oob_score_))

0.45081967213114754
0.7914240408033801


### Part 4B: Metrics
Show me each of the following model metrics for your model:
* Accuracy score
* Confusion matrix
* Log-likelihood score (sometimes called log-loss or cross-entropy)

**HINT:** For that last one, you'll need `metrics.log_loss`, which needs your predicted values to be in the form of `predict_proba`.

In [60]:
print((rfreg.oob_score_))

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 123)
logit_simple = LogisticRegression(C=1e9).fit(X_train, y_train)
logit_pred_proba = logit_simple.predict_proba(X_test)[:,1]

print(mt.confusion_matrix(y_true=y_test, y_pred=logit_pred_proba > .5))

logreg = LogisticRegression()

logreg.fit(X, y)
pred = logreg.predict(X)
log_proba = logreg.predict_proba(X)
log_proba
mt.log_loss(y, log_proba)

0.7914240408033801
[[55  8]
 [ 5 54]]


C:\Users\614318\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.1328872111174878

## Part BONUS: You
According to your model, how does it say YOU will vote? Setting this up is a bit of a challenge, so this is not mandatory.

(You can make up things about you if you want.)

In [72]:
#copied down features for my reference.

features = ['age_combined', 'educ4', 'likely', 'Male', 'race_Black',
            'race_Hispanic', 'race_Other', 'race_White', 'approve_Disapp.',
            "approve_Don't know", "genballot_Don't know", 'genballot_Reps. keep House',
            'fav_dem_Favorable', 'fav_dem_Unfavorable', 'fav_rep_Favorable', 'fav_rep_Unfavorable',
            'party_Other', 'party_Republican', 'region_North', 'region_South']

my_stats = [[1,3,4,1,0,0,0,1,1,0,1,0,0,1,0,1,0,1,1,0]]

rfreg.predict(my_stats)

# This model gives me a 22.67% change of voting Republican.

array([0.22666667])